In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
from typing import Union, Tuple, Optional, Callable
import numpy as np
import pandas as pd
import warnings
import logging
from copy import deepcopy
from joblib import Parallel, delayed, cpu_count
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
import optuna
from optuna.samplers import TPESampler, RandomSampler

from skforecast.exceptions import LongTrainingWarning
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.utils import check_backtesting_input
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.model_selection.model_selection import _get_metric

In [12]:
np.floor(100/6, dtype='int32')

TypeError: No loop matching the specified signature and casting was found for ufunc floor

In [3]:
def _create_backtesting_folds_2(
    data: Union[pd.Series, pd.DataFrame],
    window_size: int,
    initial_train_size: Union[int, None],
    test_size: int,
    externally_fitted: bool=False,
    refit: Optional[Union[bool, int]]=False,
    fixed_train_size: bool=True,
    gap: int=0,
    allow_incomplete_fold: bool=True,
    return_all_indexes: bool=False,
    verbose: bool=True
) -> list:
    """
    This function is designed to work after passing the `check_backtesting_input` 
    function from `skforecast.utils`.

    Provides train/test indices (position) to split time series data samples that
    are observed at fixed time intervals, in train/test sets. In each split, test
    indices must be higher than before.

    Three arrays are returned for each fold with the position of train, test
    including the gap, and test excluding the gap. The gap is the number of
    samples to exclude from the end of each train set before the test set. The
    test excluding the gap is the one that must be used to make evaluate the
    model. The test including the gap is provided for convenience.

    Returned indexes are not the indexes of the original time series, but the
    positional indexes of the samples in the time series. For example, if the   
    original time series is `y = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]`, the
    returned indexes for the first fold if  `test_size = 4`, `gap = 1` and 
    `initial_train_size = 2` are: `[[0, 1], [2, 3, 4, 5], [3, 4, 5]]]`. This means
    that the first fold is using the samples with positional indexes 0 and 1 in
    the time series as training set, and the samples with positional indexes 2,
    3, 4 and 5 as test set, but only the samples with positional indexes 3, 4 and
    5 should be used to evaluate the model since `gap = 1`. The second fold would
    be `[[0, 1, 2, 3], [4, 5, 6, 7], [5, 6, 7]]`, and so on.

    Each fold also provides information as to whether the Forecaster needs to 
    be trained or not.
    
    Parameters
    ----------
    data : pandas Series, pandas DataFrame
        Time series values.
    window_size : int
        Size of the window needed to create the predictors.
    initial_train_size : int, None
        Size of the training set in the first fold. If `None` or 0, the initial
        fold does not include a training set.
    test_size : int
        Size of the test set in each fold.
    externally_fitted : bool, default `False`
        Flag indicating whether the forecaster is already trained. Only used when 
        `initial_train_size` is None and `refit` is False.
    refit : bool, int, default `False`
        Whether to re-fit the forecaster in each iteration. If `refit` is an integer, 
        the Forecaster will be trained every that number of iterations.
    fixed_train_size : bool, default `True`
        If True, train size doesn't increase but moves by `steps` in each iteration.
    gap : int, default `0`
        Number of samples to be excluded after the end of each training set and 
        before the test set.
    allow_incomplete_fold : bool, default `True`
        Last fold is allowed to have a smaller number of samples than the 
        `test_size`. If `False`, the last fold is excluded.
    return_all_indexes : bool, default `False`
        If `True`, return all the indexes included in each fold. If `False`, return
        only the first and last index of each partition in each fold.
    verbose : bool, default `True`
        Print information if the folds created.

    Returns
    -------
    folds : list
        List containing the indices (position) of `y` for training, test including
        the gap, test excluding the gap for each fold, and whether fir the Forecaster.
    
    """
    
    idx = range(len(data))
    folds = []
    i = 0
    last_fold_excluded = False

    while initial_train_size + (i * test_size) + gap < len(data):

        if refit:
            # If fixed_train_size the train size doesn't increase but moves by 
            # `test_size` positions in each iteration. If False, the train size
            # increases by `test_size` in each iteration.
            train_idx_start = i * (test_size) if fixed_train_size else 0
            train_idx_end = initial_train_size + i * (test_size)
            test_idx_start = train_idx_end
        else:
            # The train size doesn't increase and doesn't move.
            train_idx_start = 0
            train_idx_end = initial_train_size
            test_idx_start = initial_train_size + i * (test_size)
            
        last_window_start = test_idx_start - window_size 
        test_idx_end = test_idx_start + gap + test_size
    
        partitions = [
            idx[train_idx_start : train_idx_end],
            idx[last_window_start : test_idx_start],
            idx[test_idx_start : test_idx_end],
            idx[test_idx_start + gap : test_idx_end]
        ]
        folds.append(partitions)
        i += 1

    if not allow_incomplete_fold:
        if len(folds[-1][2]) < test_size:
            folds = folds[:-1]
            last_fold_excluded = True

    # Replace partitions inside folds with length 0 with None
    folds = [[partition if len(partition) > 0 else None 
              for partition in fold] 
             for fold in folds]

    # Create a flag to know whether to train the forecaster
    if refit == 0:
        refit = False
        
    if isinstance(refit, bool):
        fit_forecaster = [refit]*len(folds)
        fit_forecaster[0] = True
    else:
        fit_forecaster = [False]*len(folds)
        for i in range(0, len(fit_forecaster), refit): 
            fit_forecaster[i] = True
    
    for i in range(len(folds)): 
        folds[i].append(fit_forecaster[i])
        if fit_forecaster[i] is False:
            folds[i][0] = folds[i-1][0]

    # This is done to allow parallelization. The first Forecaster fit is 
    # outside the auxiliary function.
    fit_forecaster[0] = False
    
    if verbose:
        print("Information of backtesting process")
        print("----------------------------------")
        if externally_fitted:
            print(f"An already trained forecaster is to be used. Window size: {window_size}")
        else:
            print(f"Number of observations used for initial training: {initial_train_size}")
        print(f"Number of observations used for backtesting: {len(data) - initial_train_size}")
        print(f"    Number of folds: {len(folds)}")
        print(f"    Number of steps per fold: {test_size}")
        print(f"    Number of steps to exclude from the end of each train set before test (gap): {gap}")
        if last_fold_excluded:
            print("    Last fold has been excluded because it was incomplete.")
        if len(folds[-1][3]) < test_size:
            print(f"    Last fold only includes {len(folds[-1][3])} observations.")
        print("")

        for i, fold in enumerate(folds):
            training_start    = data.index[fold[0][0]] if fold[0] is not None else None
            training_end      = data.index[fold[0][-1]] if fold[0] is not None else None
            training_length   = len(fold[0]) if fold[0] is not None else 0
            validation_start  = data.index[fold[3][0]]
            validation_end    = data.index[fold[3][-1]]
            validation_length = len(fold[3])
            print(f"Fold: {i}")
            if not externally_fitted:
                print(
                    f"    Training:   {training_start} -- {training_end}  (n={training_length})"
                )
            print(
                f"    Validation: {validation_start} -- {validation_end}  (n={validation_length})"
            )
        print("")

    if not return_all_indexes:
        # +1 to prevent iloc pandas from deleting the last observation
        folds = [
            [[fold[0][0], fold[0][-1]+1], 
             [fold[1][0], fold[1][-1]+1], 
             [fold[2][0], fold[2][-1]+1],
             [fold[3][0], fold[3][-1]+1],
             fold[4]] 
            for fold in folds
        ]

    return folds

In [8]:
from skforecast.model_selection.model_selection import _create_backtesting_folds

y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
initial_train_size = 50
gap = 5
test_size = 7
refit = False
allow_incomplete_fold = False

folds = _create_backtesting_folds_2(
            data                  = y,
            window_size           = 5,
            initial_train_size    = initial_train_size,
            test_size             = test_size,
            externally_fitted     = False,
            refit                 = 2,
            fixed_train_size      = False,
            gap                   = gap,
            allow_incomplete_fold = allow_incomplete_fold,
            return_all_indexes    = False,
            verbose               = False
        )

In [9]:
folds

[[[0, 50], [45, 50], [50, 62], [55, 62], True],
 [[0, 50], [52, 57], [57, 69], [62, 69], False],
 [[0, 64], [59, 64], [64, 76], [69, 76], True],
 [[0, 64], [66, 71], [71, 83], [76, 83], False],
 [[0, 78], [73, 78], [78, 90], [83, 90], True],
 [[0, 78], [80, 85], [85, 97], [90, 97], False],
 [[0, 92], [87, 92], [92, 100], [97, 100], True]]

In [43]:
refit = 0

if refit:
    print(2)

In [56]:
refit = 4

if isinstance(refit, bool):
    fit_forecaster = [refit]*len(folds)
else:
    fit_forecaster = [False]*len(folds)
    for i in range(0, len(fit_forecaster), refit): fit_forecaster[i] = True

for i in range(len(folds)): 
    folds[i].append(fit_forecaster[i])
    if fit_forecaster[i] is False:
        folds[i][0] = folds[i-1][0]

folds

[[[0, 91], [70, 82], [75, 82], 1, True, False, True, True, True],
 [[0, 91], [77, 89], [82, 89], 1, True, False, False, False, False],
 [[0, 91], [84, 96], [89, 96], 1, True, False, True, False, False],
 [[0, 91], [91, 100], [96, 100], 1, True, False, False, True, False]]

In [25]:
# Replace partitions inside folds with length 0 with None
folds[0].append(True)

In [72]:
aa = [False, False, False]

In [74]:
aa[0] = True
aa

[True, False, False]

In [21]:
n_boot = -1

if not isinstance(n_boot, (bool, int, np.integer)) or n_boot < 0:
    print(1)

1


# _backtesting_forecaster

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection.model_selection import _backtesting_forecaster

from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

In [20]:
%%timeit -n 200 -r 4

forecaster = ForecasterAutoreg(regressor=LinearRegression(), lags=3)

n_backtest = 12
y_train = y[:-n_backtest]

metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = None,
                                    refit = True,
                                    initial_train_size  = len(y_train),
                                    steps               = 4,
                                    metric              = 'mean_squared_error',
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    n_jobs              = 1,
                                    verbose             = False,
                                    show_progress = False
                                )

6.27 ms ± 74 µs per loop (mean ± std. dev. of 4 runs, 200 loops each)


In [18]:
%%timeit -n 200 -r 4

forecaster = ForecasterAutoreg(regressor=LinearRegression(), lags=3)

n_backtest = 12
y_train = y[:-n_backtest]

metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster          = forecaster,
                                    y                   = y,
                                    exog                = None,
                                    refit = True,
                                    initial_train_size  = len(y_train),
                                    steps               = 4,
                                    metric              = 'mean_squared_error',
                                    interval            = None,
                                    n_boot              = 500,
                                    random_state        = 123,
                                    in_sample_residuals = True,
                                    n_jobs              = 1,
                                    verbose             = False,
                                    show_progress = False
                                )

6.29 ms ± 60.9 µs per loop (mean ± std. dev. of 4 runs, 200 loops each)


## Tests

In [26]:
import pytest
import numpy as np
import pandas as pd
from pytest import approx
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection.model_selection import _backtesting_forecaster

from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

In [20]:
y.head(2)

0    0.696469
1    0.286139
dtype: float64

In [21]:
exog.head(2)

0    0.120629
1    0.826341
Name: exog, dtype: float64

In [33]:
forecaster = ForecasterAutoregDirect(
                     regressor = LinearRegression(), 
                     lags      = 3,
                     steps     = 8
                 )

forecaster.fit(y=y.iloc[:30], exog=exog.iloc[:30])

In [36]:
forecaster.predict(exog=exog.iloc[30:])

30    0.582731
31    0.449588
32    0.595278
33    0.650230
34    0.523646
35    0.468093
36    0.487597
37    0.471724
Name: pred, dtype: float64

In [37]:
0.595890 - 0.582731

0.013159000000000032

In [38]:
forecaster.predict(exog=exog.iloc[30:], steps=[4,5,6,7,8])

33    0.650230
34    0.523646
35    0.468093
36    0.487597
37    0.471724
Name: pred, dtype: float64

In [39]:
0.650230 + 0.013159000000000032

0.663389

In [42]:
np.random.seed(123)
d = {'1': np.random.rand(100), '2': np.random.rand(100), '3': np.random.rand(100)}
d

{'1': array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
        0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
        0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
        0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
        0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
        0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
        0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
        0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
        0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
        0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453,
        0.12062867, 0.8263408 , 0.60306013, 0.54506801, 0.34276383,
        0.30412079, 0.41702221, 0.68130077, 0.87545684, 0.51042234,
        0.66931378, 0.58593655, 0.6249035 , 0.67468905, 0.84234244,
        0.08319499, 0.76368284, 0.24366637, 0.19422296, 0.57245696,
        0.09571252, 0.88532683, 0.62724897,

In [49]:
rng = np.random.default_rng(seed=123)
sample_residuals = rng.choice(
                        a       = d['1'],
                        size    = 5,
                        replace = True
                    )
print(sample_residuals)
sample_residuals = rng.choice(
                        a       = d['2'],
                        size    = 5,
                        replace = True
                    )
print(sample_residuals)
sample_residuals = rng.choice(
                        a       = d['3'],
                        size    = 5,
                        replace = True
                    )
print(sample_residuals)

[0.28613933 0.19422296 0.51042234 0.42310646 0.70495883]
[0.29248941 0.98363088 0.00268806 0.86586446 0.52153306]
[0.89497829 0.01611863 0.46190876 0.35595767 0.09703816]


In [52]:
for step in ['2', '3']:
    rng = np.random.default_rng(seed=123)
    sample_residuals = rng.choice(
                            a       = d[step],
                            size    = 5,
                            replace = True
                        )
    print(sample_residuals)

[0.66662455 0.750717   0.44032788 0.66156434 0.20609573]
[0.06677444 0.30061014 0.26690581 0.57377411 0.42981334]


In [53]:
for step in ['1', '2', '3']:
    rng = np.random.default_rng(seed=123)
    sample_residuals = rng.choice(
                            a       = d[step],
                            size    = 5,
                            replace = True
                        )
    print(sample_residuals)

[0.28613933 0.19422296 0.51042234 0.42310646 0.70495883]
[0.66662455 0.750717   0.44032788 0.66156434 0.20609573]
[0.06677444 0.30061014 0.26690581 0.57377411 0.42981334]


In [34]:
forecaster.predict_bootstrapping(exog=exog.iloc[30:])

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9,...,pred_boot_490,pred_boot_491,pred_boot_492,pred_boot_493,pred_boot_494,pred_boot_495,pred_boot_496,pred_boot_497,pred_boot_498,pred_boot_499
30,0.595890,0.319955,0.594976,0.719428,0.929964,0.505866,0.682543,1.074957,0.337421,1.074957,...,0.505866,0.594976,0.594976,0.834912,0.682543,0.719428,0.337421,0.682543,0.693378,0.403944
31,0.286934,0.286934,0.661122,0.458739,0.536715,0.060349,0.804013,0.578588,0.124023,0.286934,...,0.060349,0.536715,0.571531,0.629584,0.229138,0.522168,0.551815,0.571531,0.522168,0.522168
32,0.688873,0.722722,0.917457,0.990094,0.672651,0.917457,0.618155,0.722722,0.617088,0.439513,...,0.766426,0.423266,0.696094,0.439513,0.917457,0.181711,0.563551,0.181711,0.181711,0.688873
33,0.591729,0.727680,0.735237,0.778007,0.521545,0.730467,0.725167,0.552930,0.392279,0.997500,...,0.591729,0.392279,0.997500,0.248801,0.728662,0.565366,1.065039,0.565366,0.521545,0.464614
34,0.440248,0.239002,0.652858,0.444009,0.654075,0.450979,0.923044,0.416877,0.440248,0.645430,...,0.239002,0.194533,0.654075,0.535159,0.450979,0.522948,0.416528,0.654075,0.568009,0.450979
35,0.398310,0.215374,0.402093,0.753951,0.300307,0.885939,0.398310,0.402093,0.488850,0.398310,...,0.426543,0.591517,0.644463,0.300307,0.478034,0.478034,0.885939,0.644463,0.398310,0.885939
36,0.831451,0.513211,0.299214,0.522564,0.280663,0.513211,0.750249,0.513211,0.587898,0.240795,...,0.863003,0.511131,0.240795,0.421932,0.513211,0.357115,0.587898,0.644121,0.280663,0.587898
37,0.145334,0.548837,0.675221,0.710396,0.653531,0.374671,0.268488,0.351567,0.675221,0.145334,...,0.746976,0.263568,0.145334,0.675221,0.543662,0.710396,0.145334,0.351567,0.263568,0.367503


In [35]:
forecaster.predict_bootstrapping(steps=[4,5,6,7,8], exog=exog.iloc[30:])

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9,...,pred_boot_490,pred_boot_491,pred_boot_492,pred_boot_493,pred_boot_494,pred_boot_495,pred_boot_496,pred_boot_497,pred_boot_498,pred_boot_499
33,1.065039,0.565366,0.271832,0.730467,0.778007,0.536698,0.997500,0.591729,0.735237,0.591729,...,0.536698,0.271832,0.271832,0.552930,0.997500,0.730467,0.735237,0.997500,0.521545,0.248801
34,0.194533,0.194533,0.416877,0.652858,0.450979,0.239002,0.654075,0.522948,0.568009,0.194533,...,0.239002,0.450979,0.535159,0.473526,0.645430,0.923044,0.444009,0.535159,0.923044,0.923044
35,0.501507,0.300307,0.591517,0.332730,0.406936,0.591517,0.644463,0.300307,0.753951,0.645406,...,0.488850,0.398310,0.426543,0.645406,0.591517,0.124407,0.402093,0.124407,0.124407,0.501507
36,0.466250,0.335404,0.831451,0.280663,0.421932,0.357115,0.528976,0.121076,0.513211,0.511131,...,0.466250,0.513211,0.511131,0.240795,0.587898,0.750249,0.369764,0.750249,0.421932,0.863003
37,0.351567,0.543148,0.374671,0.017550,0.145334,0.807425,0.548837,0.376153,0.351567,0.746976,...,0.543148,0.586538,0.145334,0.675221,0.807425,0.268488,0.710396,0.145334,0.464328,0.807425


In [13]:
forecaster.predict(steps=[3,4,5], exog=exog.iloc[22:25])

ValueError: `exog` must have at least as many values as the distance to the maximum step predicted, 5.

In [30]:
forecaster = ForecasterAutoregDirect(
                    regressor = LinearRegression(), 
                    lags      = 3,
                    steps     = 8
                )

n_backtest = 20
y_train = y[:-n_backtest]

metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster            = forecaster,
                                    y                     = y,
                                    exog                  = exog,
                                    refit                 = False,
                                    initial_train_size    = len(y_train),
                                    gap                   = 3,
                                    allow_incomplete_fold = True,
                                    steps                 = 5,
                                    metric                = 'mean_squared_error',
                                    interval              = [5, 95],
                                    n_boot                = 500,
                                    random_state          = 123,
                                    in_sample_residuals   = True,
                                    verbose               = False,
                                    show_progress=False
                                )

8
8
8
8
8
8
5
5


In [29]:
backtest_predictions

,pred,lower_bound,upper_bound
33,0.650230,0.271832,1.065039
34,0.523646,0.194533,0.882655
35,0.468093,0.124407,0.753951
36,0.487597,0.240795,0.863003
37,0.471724,0.145334,0.807425
38,0.575858,0.197460,0.990668
39,0.535929,0.206816,0.894938
40,0.489951,0.146265,0.775809
41,0.452713,0.205911,0.828119
42,0.495194,0.168803,0.830894


In [31]:
backtest_predictions

,pred,lower_bound,upper_bound
30,0.582731,0.274546,0.929964
31,0.449588,0.124023,0.804013
32,0.595278,0.181711,0.990094
33,0.650230,0.248801,1.065039
34,0.523646,0.194533,0.923044
35,0.468093,0.124407,0.885939
36,0.487597,0.240795,0.863003
37,0.471724,0.138944,0.746976
35,0.411027,0.102842,0.758259
36,0.567017,0.241452,0.921442


In [38]:
forecaster = ForecasterAutoregDirect(
                     regressor = Ridge(random_state=123), 
                     lags      = 3,
                     steps     = 8
                 )

n_backtest = 35
y_train = y[:-n_backtest]

metric, backtest_predictions = backtesting_forecaster(
                                    forecaster            = forecaster,
                                    y                     = y,
                                    exog                  = exog,
                                    refit                 = 2,
                                    initial_train_size    = len(y_train),
                                    fixed_train_size      = False,
                                    gap                   = 0,
                                    allow_incomplete_fold = True,
                                    steps                 = 2,
                                    metric                = 'mean_squared_error',
                                    interval              = [5, 95],
                                    n_boot                = 500,
                                    random_state          = 123,
                                    in_sample_residuals   = True,
                                    verbose               = False
                                )

  0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\model_selection.py:525: LongTrainingWarning: The forecaster will be fit 72.0 times (9.0 folds * 8 regressors). This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(


In [14]:
metric

0.06099110404144631

In [12]:
backtest_predictions.to_numpy()

array([[0.55616986, 0.15288789, 0.89198752],
       [0.48751797, 0.14866438, 0.83169303],
       [0.57764391, 0.17436194, 0.91346157],
       [0.51298667, 0.17413308, 0.85716173],
       [0.47430051, 0.0796644 , 0.82587748],
       [0.49192271, 0.14609696, 0.95959395],
       [0.52213783, 0.12750172, 0.8737148 ],
       [0.54492575, 0.1991    , 1.012597  ],
       [0.52501537, 0.13641764, 0.86685356],
       [0.4680474 , 0.08515461, 0.81792677],
       [0.51059498, 0.12199725, 0.85243317],
       [0.53067132, 0.14777853, 0.88055069],
       [0.4430938 , 0.0509291 , 0.69854202],
       [0.49911716, 0.1231365 , 0.8711497 ],
       [0.44546347, 0.05329877, 0.70091169],
       [0.46530749, 0.08932683, 0.83734003],
       [0.46901878, 0.08173407, 0.82098555],
       [0.55371362, 0.14618224, 0.98199137],
       [0.60759064, 0.22030593, 0.9595574 ],
       [0.50415336, 0.09662198, 0.93243111]])

In [35]:
y_with_index = y.copy()
y_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')
exog_with_index = exog.copy()
exog_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')

forecaster = ForecasterAutoreg(regressor=Ridge(random_state=123), lags=3)

metric, backtest_predictions = backtesting_forecaster(
                                    forecaster            = forecaster,
                                    y                     = y_with_index,
                                    exog                  = exog_with_index,
                                    refit                 = 3,
                                    initial_train_size    = len(y_with_index) - 20,
                                    fixed_train_size      = True,
                                    gap                   = 3,
                                    allow_incomplete_fold = False,
                                    steps                 = 4,
                                    metric                = 'mean_squared_error',
                                    interval              = [5, 95],
                                    n_boot                = 500,
                                    random_state          = 123,
                                    in_sample_residuals   = True,
                                    verbose               = True,
                                    n_jobs                = 1
                                )
backtest_predictions = backtest_predictions.asfreq('D')

NameError: name 'y' is not defined

In [33]:
metric

0.060991643719298785

In [36]:
forecaster.fit(y=y_with_index.iloc[:30], exog=exog_with_index[:30])

In [37]:
forecaster.predict(7, exog=exog_with_index.iloc[30:37]).tail(4)

2022-02-03    0.518786
2022-02-04    0.492698
2022-02-05    0.493804
2022-02-06    0.514675
Freq: D, Name: pred, dtype: float64

In [39]:
forecaster.predict(7, last_window=y_with_index.iloc[:34], 
                   exog=exog_with_index.iloc[34:42]).tail(4)

2022-02-07    0.526900
2022-02-08    0.517320
2022-02-09    0.512903
2022-02-10    0.503343
Freq: D, Name: pred, dtype: float64

In [40]:
forecaster.predict(7, last_window=y_with_index.iloc[:38], 
                   exog=exog_with_index.iloc[38:45]).tail(4)

2022-02-11    0.501715
2022-02-12    0.509469
2022-02-13    0.502004
2022-02-14    0.504360
Freq: D, Name: pred, dtype: float64

In [44]:
forecaster.fit(y=y_with_index.iloc[12:42], exog=exog_with_index[12:42])
forecaster.predict(7, exog=exog_with_index.iloc[42:49]).tail(4)

2022-02-15    0.453419
2022-02-16    0.526217
2022-02-17    0.504778
2022-02-18    0.522353
Freq: D, Name: pred, dtype: float64

In [46]:
(backtest_predictions)

,pred,lower_bound,upper_bound
2022-02-03,0.518786,0.173240,0.877714
2022-02-04,0.492698,0.152233,0.868473
2022-02-05,0.493804,0.140035,0.812267
2022-02-06,0.514675,0.182259,0.915069
2022-02-07,0.526900,0.181354,0.885829
2022-02-08,0.517320,0.176855,0.893095
2022-02-09,0.512903,0.159134,0.831365
2022-02-10,0.503343,0.170928,0.903737
2022-02-11,0.501715,0.156169,0.860643
2022-02-12,0.509469,0.169004,0.885245


In [35]:
backtest_predictions.to_numpy()

array([[0.51878642, 0.17324039, 0.87771447],
       [0.49269791, 0.15223278, 0.86847335],
       [0.49380441, 0.14003517, 0.81226679],
       [0.51467463, 0.18225936, 0.91506854],
       [0.52690045, 0.18135443, 0.8858285 ],
       [0.51731996, 0.17685482, 0.8930954 ],
       [0.51290311, 0.15913388, 0.83136549],
       [0.50334306, 0.17092779, 0.90373697],
       [0.50171526, 0.15616923, 0.86064331],
       [0.50946908, 0.16900395, 0.88524452],
       [0.50200357, 0.14823433, 0.82046595],
       [0.50436041, 0.17194514, 0.90475432],
       [0.4534189 , 0.09320851, 0.83561058],
       [0.52621695, 0.17305963, 0.91133042],
       [0.50477802, 0.10690002, 0.88750077],
       [0.52235258, 0.1700762 , 0.91128311]])

## Multiseries

In [3]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import backtesting_forecaster_multivariate

# Fixtures
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series

In [30]:
series_with_index = series.copy()
series_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')
exog_with_index = series['l1'].rename('exog_1').copy()
exog_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')

forecaster = ForecasterAutoregMultiVariate(
                    regressor = Ridge(random_state=123),
                    level     = 'l1',
                    lags      = 2,
                    steps     = 7
                )

refit = 3
n_validation = 30
steps = 4
gap = 3

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series_with_index,
                                            steps                 = steps,
                                            levels                = 'l1',
                                            metric                = 'mean_absolute_error',
                                            initial_train_size    = len(series) - n_validation,
                                            gap                   = gap,
                                            allow_incomplete_fold = False,
                                            refit                 = refit,
                                            fixed_train_size      = False,
                                            exog                  = exog_with_index,
                                            interval              = [5, 95],
                                            n_boot                = 100,
                                            random_state          = 123,
                                            in_sample_residuals   = True,
                                            verbose               = True
                                        )

Information of backtesting process
----------------------------------
Number of observations used for initial training: 20
Number of observations used for backtesting: 30
    Number of folds: 6
    Number of steps per fold: 4
    Number of steps to exclude from the end of each train set before test (gap): 3
    Last fold has been excluded because it was incomplete.

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-01-20 00:00:00  (n=20)
    Validation: 2022-01-24 00:00:00 -- 2022-01-27 00:00:00  (n=4)
Fold: 1
    Training:   2022-01-01 00:00:00 -- 2022-01-20 00:00:00  (n=20)
    Validation: 2022-01-28 00:00:00 -- 2022-01-31 00:00:00  (n=4)
Fold: 2
    Training:   2022-01-01 00:00:00 -- 2022-01-20 00:00:00  (n=20)
    Validation: 2022-02-01 00:00:00 -- 2022-02-04 00:00:00  (n=4)
Fold: 3
    Training:   2022-01-01 00:00:00 -- 2022-02-01 00:00:00  (n=32)
    Validation: 2022-02-05 00:00:00 -- 2022-02-08 00:00:00  (n=4)
Fold: 4
    Training:   2022-01-01 00:00:00 -- 2022-02-01 00:00:00 

  0%|          | 0/6 [00:00<?, ?it/s]

In [31]:
metrics_levels.to_numpy()

array([['l1', 0.10066454067329249]], dtype=object)

In [34]:
len(backtest_predictions)

24

## Direct

In [64]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import norm
import pytest
import numpy as np
import pandas as pd
from pytest import approx
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection.model_selection import _backtesting_forecaster
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries

# Fixtures
# from skforecast.ForecasterAutoregMultiVariate.tests.fixtures_ForecasterAutoregMultiVariate import series
# from skforecast.ForecasterAutoregMultiVariate.tests.fixtures_ForecasterAutoregMultiVariate import exog
# from skforecast.ForecasterAutoregMultiVariate.tests.fixtures_ForecasterAutoregMultiVariate import exog_predict

from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series

transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [86]:
    series_with_index = series.copy()
    series_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')
    exog_with_index = series['l1'].rename('exog_1').copy()
    exog_with_index.index = pd.date_range(start='2022-01-01', periods=50, freq='D')

    forecaster = ForecasterAutoregMultiVariate(
                     regressor = Ridge(random_state=123),
                     level     = 'l1',
                     lags      = 2,
                     steps     = 7
                 )

    refit = 3
    n_validation = 30
    steps = 4
    gap = 3

    metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                               forecaster            = forecaster,
                                               series                = series_with_index,
                                               steps                 = steps,
                                               levels                = ['l1'],
                                               metric                = 'mean_absolute_error',
                                               initial_train_size    = len(series_with_index) - n_validation,
                                               gap                   = gap,
                                               allow_incomplete_fold = False,
                                               refit                 = refit,
                                               fixed_train_size      = False,
                                               exog                  = exog_with_index,
                                               interval              = [5, 95],
                                               n_boot                = 100,
                                               random_state          = 123,
                                               in_sample_residuals   = True,
                                               verbose               = False
                                           )

  0%|          | 0/6 [00:00<?, ?it/s]

In [87]:
metrics_levels.to_numpy()

array([['l1', 0.10066454067329249]], dtype=object)

In [88]:
backtest_predictions.to_numpy()

array([[0.53203113, 0.27318916, 0.69931226],
       [0.62129188, 0.46220068, 0.85870486],
       [0.40145922, 0.17189084, 0.59797159],
       [0.38821275, 0.17098498, 0.61456487],
       [0.37636995, 0.11752798, 0.54365108],
       [0.36599243, 0.20690123, 0.60340541],
       [0.47654862, 0.24698023, 0.67306099],
       [0.33725755, 0.12002978, 0.56360967],
       [0.46122163, 0.20237966, 0.62850276],
       [0.47541934, 0.31632814, 0.71283232],
       [0.50601999, 0.27645161, 0.70253236],
       [0.39293452, 0.17570675, 0.61928664],
       [0.43793405, 0.27695885, 0.5990429 ],
       [0.52600886, 0.38167775, 0.72224797],
       [0.70882239, 0.55319058, 0.83377003],
       [0.70063755, 0.51962777, 0.85415566],
       [0.48631217, 0.32533698, 0.64742102],
       [0.52248417, 0.37815306, 0.71872328],
       [0.30010035, 0.14446853, 0.42504798],
       [0.43342987, 0.25242008, 0.58694797],
       [0.42479569, 0.2638205 , 0.58590454],
       [0.68566159, 0.54133048, 0.8819007 ],
       [0.